In [2]:
from vpython import *
import serial
import string
import math

from time import time

In [52]:
# Main scene
scene=canvas(title="Arduino Nano + LSM9DS1 w/ Madgwick",
             width=500,
             height=300,
             center=vector(0,0,.20),
             #range=1.2,
             forward=vector(-.4,-.8,-.25),
             up=vector(0,0,1),
             userpan = True)

# Second scene (Roll, Pitch, Yaw)
scene2 = canvas(title='Arduino Nano + LSM9DS1 w/ Madgwick',
                 x=0,
                 y=0,
                 width=500,
                 height=300,
                 center=vector(.1,0,.5),
                 background=vector(0,0,0),
                 userpan = True,
                 range=.7)

# Main scene objects
# Reference axis (x,y,z)
arrow(canvas=scene,color=color.green,axis=vector(1,0,0), shaftwidth=0.05, fixedwidth=1)
arrow(canvas=scene,color=color.green,axis=vector(0,1,0), shaftwidth=0.05, fixedwidth=1)
arrow(canvas=scene,color=color.green,axis=vector(0,0,1), shaftwidth=0.05 , fixedwidth=1)
# labels
#label(canvas=scene,pos=vector(0,0,0.8),text="Arduino Nano + LSM9DS1 w/ Madgwick",box=0,opacity=0)
label(canvas=scene,pos=vector(1.1,0,0),text="X",box=0,opacity=0)
label(canvas=scene,pos=vector(0,1.1,0),text="Y",box=0,opacity=0)
label(canvas=scene,pos=vector(0,0,1.1),text="Z",box=0,opacity=0)
# IMU object
platform = box(canvas=scene,up=vector(0,0,0),axis=vector(1,0,0),length=1, height=0.05, width=1, color=color.blue)
p_line = box(canvas=scene,up=vector(0,0,0),axis=vector(1,0,0),length=1,height=0.08,width=0.1,color=color.yellow)
plat_arrow = arrow(canvas=scene,up=vector(0,0,0),color=color.green,axis=vector(1,0,0), shaftwidth=0.06, fixedwidth=1)

#Second Scene
#Roll, Pitch, Yaw
cil_roll = cylinder(canvas=scene2,pos=vector(-0.4,0,0),axis=vector(0.2,0,0),radius=0.01)
cil_roll2 = cylinder(canvas=scene2,pos=vector(-0.4,0,0),axis=vector(-0.2,0,0),radius=0.01,color=color.red)
cil_pitch = cylinder(canvas=scene2,pos=vector(0.1,0,0),axis=vector(0.2,0,0),radius=0.01,color=color.green)
cil_pitch2 = cylinder(canvas=scene2,pos=vector(0.1,0,0),axis=vector(-0.2,0,0),radius=0.01,color=color.green)
#cil_course = cylinder(pos=vector(0.6,0,0),axis=(0.2,0,0),radius=0.01,color=color.blue)
#cil_course2 = cylinder(pos=vector(0.6,0,0),axis=(-0.2,0,0),radius=0.01,color=color.blue)
arrow_course = arrow(canvas=scene2,pos=vector(0.6,0,0),color=color.cyan,axis=vector(-0.2,0,0), shaftwidth=0.02, fixedwidth=1)

#Roll,Pitch,Yaw labels
label(canvas=scene2,pos=vector(-0.4,0.5,0),text="Pitch",box=0,opacity=0)
label(canvas=scene2,pos=vector(0.1,0.5,0),text="Roll",box=0,opacity=0)
label(canvas=scene2,pos=vector(0.55,0.5,0),text="Yaw",box=0,opacity=0)
label(canvas=scene2,pos=vector(0.6,0.22,0),text="N",box=0,opacity=0,color=color.yellow)
label(canvas=scene2,pos=vector(0.6,-0.22,0),text="S",box=0,opacity=0,color=color.yellow)
label(canvas=scene2,pos=vector(0.38,0,0),text="W",box=0,opacity=0,color=color.yellow)
label(canvas=scene2,pos=vector(0.82,0,0),text="E",box=0,opacity=0,color=color.yellow)
label(canvas=scene2,pos=vector(0.75,0.15,0),height=7,text="NE",box=0,color=color.yellow)
label(canvas=scene2,pos=vector(0.45,0.15,0),height=7,text="NW",box=0,color=color.yellow)
label(canvas=scene2,pos=vector(0.75,-0.15,0),height=7,text="SE",box=0,color=color.yellow)
label(canvas=scene2,pos=vector(0.45,-0.15,0),height=7,text="SW",box=0,color=color.yellow)

L1 = label(canvas=scene2,pos=vector(-0.4,0.4,0),text="-",box=0,opacity=0)
L2 = label(canvas=scene2,pos=vector(0.1,0.4,0),text="-",box=0,opacity=0)
L3 = label(canvas=scene2,pos=vector(0.55,0.4,0),text="-",box=0,opacity=0)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [54]:
#Serial Connection for Arduino
ser = serial.Serial(port='COM3',baudrate=115200, timeout=1)

#Initialize variables
roll=0
pitch=0
yaw=0

#Degrees to radians
grad2rad = 3.141592/180.0

while True:
    line = ser.readline()
    #Decode bytes into string
    decoded = str(line, 'utf-8')
    #print(decoded)
    size = len(decoded)
    #slice string to remove last 2 characters from string
    mod_string = decoded[:size - 2]
    #print(mod_string)
    words = str.split(mod_string,",")    # Fields split
    print(float(words[0]),float(words[1]),float(words[2]))
    print(len(words))
    
    rate(1000)
    if len(words) > 2:
        print('If statement, yo')
        try:
            yaw = float(words[0])*grad2rad
            pitch = float(words[1])*grad2rad
            roll = float(words[2])*grad2rad
            print(yaw,pitch,roll)
        except:
            print("Invalid line")
        
        axis=vector(cos(pitch)*cos(yaw),-cos(pitch)*sin(yaw),sin(pitch)) 
        up=vector(sin(roll)*sin(yaw)+cos(roll)*sin(pitch)*cos(yaw),sin(roll)*cos(yaw)-cos(roll)*sin(pitch)*sin(yaw),-cos(roll)*cos(pitch))
        platform.axis=axis
        platform.up=up
        #platform.length=1.0
        #platform.width=0.65
        plat_arrow.axis=axis
        plat_arrow.up=up
        #plat_arrow.length=0.8
        p_line.axis=axis
        p_line.up=up
        cil_roll.axis=vector(0.2*cos(roll),0.2*sin(roll),0)
        cil_roll2.axis=vector(-0.2*cos(roll),-0.2*sin(roll),0)
        cil_pitch.axis=vector(0.2*cos(pitch),0.2*sin(pitch),0)
        cil_pitch2.axis=vector(-0.2*cos(pitch),-0.2*sin(pitch),0)
        arrow_course.axis=vector(0.2*sin(yaw),0.2*cos(yaw),0)
        L1.text = str(float(words[0]))
        L2.text = str(float(words[1]))
        L3.text = str(float(words[2]))        
ser.close

185.26 -0.2 -1.44
3
If statement, yo
3.233396299555556 -0.0034906577777777783 -0.025132736000000003
185.87 -0.23 -1.52
3
If statement, yo
3.244042805777778 -0.004014256444444445 -0.026528999111111114
186.47 -0.27 -1.58
3
If statement, yo
3.2545147791111115 -0.0047123880000000005 -0.027576196444444447
187.08 -0.3 -1.62
3
If statement, yo
3.265161285333334 -0.005235986666666667 -0.028274328000000005
187.68 -0.35 -1.65
3
If statement, yo
3.275633258666667 -0.006108651111111111 -0.028797926666666668
188.28 -0.4 -1.65
3
If statement, yo
3.286105232 -0.0069813155555555565 -0.028797926666666668
188.88 -0.4 -1.64
3
If statement, yo
3.2965772053333335 -0.0069813155555555565 -0.02862339377777778
189.47 -0.45 -1.62
3
If statement, yo
3.306874645777778 -0.007853980000000002 -0.028274328000000005
190.01 -0.47 -1.58
3
If statement, yo
3.316299421777778 -0.008203045777777779 -0.027576196444444447
190.59 -0.52 -1.54
3
If statement, yo
3.3264223293333335 -0.009075710222222224 -0.026878064888888893
191.

If statement, yo
4.132938808888889 0.3221877128888889 0.20682147333333334
234.76 11.4 12.61
3
If statement, yo
4.097334099555556 0.19896749333333336 0.2200859728888889
232.86 5.35 13.6
3
If statement, yo
4.064172850666667 0.09337509555555555 0.2373647288888889
231.4 -1.09 15.77
3
If statement, yo
4.038691048888889 -0.019024084888888893 0.27523836577777777
230.92 -0.35 16.29
3
If statement, yo
4.0303134702222225 -0.006108651111111111 0.284314076
229.77 1.01 16.76
3
If statement, yo
4.010242188 0.017627821777777778 0.29251712177777783
228.18 -2.53 18.17
3
If statement, yo
3.9824914586666673 -0.04415682088888889 0.31712625911111114
228.24 -2.23 17.63
3
If statement, yo
3.9835386560000003 -0.03892083422222223 0.3077014831111111
228.5 -4.39 17.15
3
If statement, yo
3.9880765111111116 -0.07661993822222223 0.29932390444444446
228.07 -7.23 16.13
3
If statement, yo
3.9805715968888893 -0.12618727866666668 0.28152154977777777
223.42 -13.91 15.9
3
If statement, yo
3.899413803555556 -0.242775248444

248.7 -5.93 10.47
3
If statement, yo
4.340632946666667 -0.10349800311111111 0.18273593466666668
252.57 -13.28 10.2
3
If statement, yo
4.408177174666667 -0.23177967644444444 0.17802354666666667
258.1 -9.14 9.56
3
If statement, yo
4.504693862222223 -0.15952306044444448 0.1668534417777778
265.97 -8.85 9.4
3
If statement, yo
4.642051245777779 -0.15446160666666667 0.16406091555555558
269.01 -5.79 9.86
3
If statement, yo
4.695109244 -0.10105454266666668 0.17208942844444444
275.15 -1.92 12.18
3
If statement, yo
4.802272437777778 -0.033510314666666666 0.21258105866666668
279.38 0.29 11.97
3
If statement, yo
4.876099849777778 0.0050614537777777776 0.20891586800000003
281.04 1.44 12.75
3
If statement, yo
4.905072309333334 0.025132736000000003 0.22252943333333336
281.26 1.5 12.36
3
If statement, yo
4.908912032888889 0.026179933333333336 0.21572265066666668
281.65 2.04 12.54
3
If statement, yo
4.915718815555556 0.03560470933333334 0.21886424266666668
280.67 2.87 13.24
3
If statement, yo
4.89861459

230.74 -15.14 14.18
3
If statement, yo
4.027171878222223 -0.2642427937777778 0.24748763644444446
228.99 -7.17 14.07
3
If statement, yo
3.996628622666667 -0.12514008133333335 0.24556777466666668
226.21 2.66 7.07
3
If statement, yo
3.948108479555556 0.04642574844444445 0.12339475244444446
226.0 2.78 -2.84
3
If statement, yo
3.9444432888888894 0.048520143111111115 -0.04956734044444445
226.15 -0.62 -8.16
3
If statement, yo
3.9470612822222226 -0.010821039111111113 -0.14241883733333335
228.26 -6.39 -2.91
3
If statement, yo
3.983887721777778 -0.111526516 -0.05078907066666667
228.97 -20.04 3.64
3
If statement, yo
3.996279556888889 -0.34976390933333334 0.06352997155555556
229.27 -20.95 5.55
3
If statement, yo
4.0015155435555565 -0.3656464022222222 0.09686575333333333
229.24 -16.26 6.4
3
If statement, yo
4.00099194488889 -0.2837904773333334 0.1117010488888889
227.78 -15.82 3.0
3
If statement, yo
3.9755101431111113 -0.2761110302222223 0.05235986666666667
229.94 -17.0 1.96
3
If statement, yo
4.013

248.49 -10.25 17.28
3
If statement, yo
4.336967756000001 -0.17889621111111112 0.30159283200000003
248.37 -20.11 6.19
3
If statement, yo
4.334873361333334 -0.35098563955555556 0.10803585822222224
250.0 -27.18 -3.2
3
If statement, yo
4.363322222222223 -0.47438039200000004 -0.05585052444444445
252.07 -29.49 -8.03
3
If statement, yo
4.399450530222222 -0.5146974893333334 -0.14014990977777778
254.58 -31.43 -12.17
3
If statement, yo
4.443258285333334 -0.5485568697777778 -0.2124065257777778
255.89 -32.81 -14.85
3
If statement, yo
4.466122093777778 -0.5726424084444446 -0.25918134000000004
256.51 -33.31 -15.81
3
If statement, yo
4.476943132888889 -0.581369052888889 -0.27593649733333336
256.87 -32.99 -15.61
3
If statement, yo
4.4832263168888895 -0.5757840004444446 -0.27244583955555557
256.51 -30.15 -10.88
3
If statement, yo
4.476943132888889 -0.52621666 -0.18989178311111113
255.43 -26.12 -1.49
3
If statement, yo
4.45809358088889 -0.4558799057777778 -0.026005400444444448
256.1 -20.98 11.62
3
If st

244.91 -14.8 2.96
3
If statement, yo
4.274484981777778 -0.2583086755555556 0.051661735111111114
246.71 5.47 4.25
3
If statement, yo
4.305900901777778 0.09546949022222223 0.07417647777777778
248.22 13.5 4.47
3
If statement, yo
4.332255368 0.23561940000000003 0.07801620133333334
248.27 14.06 4.08
3
If statement, yo
4.333128032444445 0.2453932417777778 0.07120941866666668
246.8 -0.74 2.22
3
If statement, yo
4.307471697777778 -0.012915433777777778 0.03874630133333334
246.78 -14.4 1.99
3
If statement, yo
4.3071226320000005 -0.25132736000000006 0.03473204488888889
246.5 -21.42 1.73
3
If statement, yo
4.302235711111112 -0.3738494480000001 0.03019418977777778
246.98 -23.04 1.48
3
If statement, yo
4.310613289777778 -0.40212377600000004 0.025830867555555557
248.27 -14.41 3.28
3
If statement, yo
4.333128032444445 -0.2515018928888889 0.05724678755555556
250.02 -0.51 4.94
3
If statement, yo
4.363671288000001 -0.008901177333333335 0.08621924711111112
250.23 6.13 4.39
3
If statement, yo
4.36733647866

If statement, yo
4.654617613777778 -0.02565633466666667 0.02076941377777778
266.84 -1.37 1.12
3
If statement, yo
4.657235607111111 -0.023911005777777782 0.01954768355555556
267.0 -1.27 1.07
3
If statement, yo
4.660028133333333 -0.022165676888888892 0.018675019111111114
267.16 -1.17 1.01
3
If statement, yo
4.662820659555557 -0.020420348 0.017627821777777778
267.31 -1.13 0.96
3
If statement, yo
4.66543865288889 -0.019722216444444444 0.016755157333333333
267.47 -1.05 0.91
3
If statement, yo
4.668231179111112 -0.018325953333333336 0.01588249288888889
267.61 -0.97 0.86
3
If statement, yo
4.670674639555556 -0.016929690222222224 0.015009828444444446
267.75 -0.91 0.82
3
If statement, yo
4.673118100000001 -0.01588249288888889 0.01431169688888889
267.89 -0.85 0.78
3
If statement, yo
4.675561560444445 -0.014835295555555556 0.013613565333333334
268.03 -0.78 0.74
3
If statement, yo
4.678005020888889 -0.013613565333333334 0.012915433777777778
268.21 -0.73 0.69
3
If statement, yo
4.681146612888889 -0

ValueError: could not convert string to float: 